# Introduction to NLP Fundamentals in Tensorflow

Natural Language Process (NLP) is the technology used to aid computers to undestand human's natural language. (could be sequences text or speech)

Another common term for NLP problem is sequence to sequence problems (seq2seq).

In [ ]:
# Import libraries
import random
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split

print(f'Tensorflow Version: {tf.__version__}')

Tensorflow Version: 2.12.0


## Check for GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-306e31d0-1eed-59fc-8896-b74fbdcfdd0e)


## Get the helper function and import some functions

In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_histories

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster)

Kaggle source here for competition: https://www.kaggle.com/competitions/nlp-getting-started

In [ ]:
# Download data as zip
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-08-20 19:36:38--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 172.253.118.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K   701KB/s    in 0.8s    

2023-08-20 19:36:39 (701 KB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [ ]:
# Unzip data
zip_ref = zipfile.ZipFile('nlp_getting_started.zip')
zip_ref.extractall()
zip_ref.close()

## Becoming one with the data

We got the data and now, we need to understand our data that's wht lets visualize the text data.

In [ ]:
# Read the data and turn it pandas dataframe
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
# Check out the first 5 row
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Get the info from train dataset
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
# Check the text row
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [ ]:
# Shuffle train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [ ]:
# Check the shuffled data
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# How does look like test data?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples of each class?
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# How many total samples?
print(f'Train total samples: {len(train_df_shuffled)}')
print(f'Test total samples: {len(test_df)}')

Train total samples: 7613
Test total samples: 3263


In [ ]:
# Let's visualize the some random training examples
random_samples = random.sample(range(0, len(train_df_shuffled)), 5)

for i in random_samples:
    target = train_df_shuffled['target'][i]
    text = train_df_shuffled['text'][i]
    print(f'Target: {target}', '(Real Diaster)' if target == 1 else '(Not Real Diaster)')
    print(f'Text: {text}', '\n')

Target: 0 (Not Real Diaster)
Text: World Annihilation vs Self Transformation http://t.co/pyehwodWun Aliens Attack to Exterminate Humans http://t.co/pB2N77nSKz 

Target: 1 (Real Diaster)
Text: Vladimir Putin Issues Major Warning But Is It Too Late To Escape Armageddon?
http://t.co/gBxafy1m1C 

Target: 1 (Real Diaster)
Text: Party for Bestival crash victim Michael Molloy on what would have been his 21st http://t.co/BIkR8zzbhA 

Target: 0 (Not Real Diaster)
Text: The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.' 

Target: 0 (Not Real Diaster)
Text: Not being able to touch anything or anyone in Penneys without being electrocuted ?? 



### Split data into training and validation sets

In [ ]:
# Use train_test_split to split our data
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before build a model is to convert text into the numbers.

There are few ways to do this:
* Tokenization - direct mapping of token (a token could be a word or a characher) to number
* Embedding - create a matrix of feature vektor for each token (the size of the feature vektor can be defined and this embedding can be learned)

### Text vektorization (tokenization)

TensorFlow Vectorization Docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [ ]:
# Use the default text vektorization parameters
text_vektorizer = TextVectorization(max_tokens=None, # How many words in the sentence (automatically add <OOV> OOV:Out of word)
                                    standardize='lower_and_strip_punctuation', # lower case the word and delete punctuations
                                    split='whitespace', # Split the sentence with whitespace
                                    ngrams=None, # Create groups of n-words?
                                    output_mode='int', # How to map tokens to numbers
                                    output_sequence_length=None, # How long do you want your sequences to be?
                                    pad_to_max_tokens=False)

In [ ]:
# Find the average number of tokens (words) in the training tweets
total_words = sum([len(i.split()) for i in train_sentences])
total_sentences = len(train_sentences)
round(total_words / total_sentences)

15

In [ ]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vektorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length,
                                    pad_to_max_tokens=True)

In [ ]:
# Fit the text vectorizer to the training text
text_vektorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vektorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print('Random sentence:\n', random_sentence, '\n')
print('Vectorized version:\n', text_vektorizer([random_sentence]))

Random sentence:
 Governor Allows Parole for School Bus Hijacker http://t.co/JwfiiLJ8Wr liveleakfun ? http://t.co/IONWArVRFy 

Vectorized version:
 tf.Tensor(
[[1080 1228  932   10  185  352  545    1    1    1    0    0    0    0
     0]], shape=(1, 15), dtype=int64)


In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vektorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocabulary: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocabulary: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding layer

Tensorflow Embedding Docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we mostly care about the Embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                                      output_dim=128,
                                      input_length=max_length) # how long is each input
embedding

In [ ]:
# Get a random sentence form the training set
random_sentence = random.choice(train_sentences)
print(f'Random sentence:\n', random_sentence, '\n')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vektorizer([random_sentence]))
print(f'Embedding version:\n', sample_embed)

Random sentence:
 Man my stomach feel like a tsunami ??. 

Embedding version:
 tf.Tensor(
[[[-0.03749081  0.02590379  0.02550626 ... -0.02854614  0.02234951
    0.03952974]
  [ 0.02603719 -0.00519217 -0.02377347 ...  0.01412911  0.01491584
    0.01299771]
  [ 0.02904962 -0.03735137  0.0121748  ...  0.04968903  0.00674528
    0.03651105]
  ...
  [ 0.03210877  0.03341455  0.0443308  ...  0.03361105 -0.0164877
   -0.01774064]
  [ 0.03210877  0.03341455  0.0443308  ...  0.03361105 -0.0164877
   -0.01774064]
  [ 0.03210877  0.03341455  0.0443308  ...  0.03361105 -0.0164877
   -0.01774064]]], shape=(1, 15, 128), dtype=float32)


In [ ]:
# Check out the single token's embedding
random_sentence, sample_embed[0][0], sample_embed[0][0].shape

('Man my stomach feel like a tsunami ??.',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03749081,  0.02590379,  0.02550626, -0.0328787 ,  0.01051283,
         0.00407202, -0.00423402, -0.02814646,  0.01450974,  0.01575614,
         0.04080807,  0.02057245, -0.0186395 ,  0.02824769,  0.03497573,
        -0.01297732, -0.00548046,  0.01258716, -0.01277789, -0.04057152,
        -0.00854657, -0.03968408, -0.00550578, -0.01156821,  0.01015099,
         0.00642205, -0.01243105, -0.04067893,  0.0420012 , -0.03525947,
         0.04886505,  0.01303501,  0.02521045,  0.01129233, -0.01043051,
        -0.03646414, -0.04435148,  0.00341837,  0.00809251,  0.03266085,
         0.04431802, -0.02994536,  0.00866849, -0.01381986,  0.02374027,
         0.04415778,  0.034538  , -0.03766326,  0.00252801,  0.01757133,
         0.02430331,  0.00404018,  0.00739849, -0.00011872, -0.00665873,
        -0.00351965, -0.03985617, -0.00493623, -0.01998878,  0.04892293,
        -0.01007959, -0.02568458

## Modelling a text dataset (running a series of experiments)

We converted our text data into numbers with vectorization and embedding, now it is time to create models. Here are the models we will create step by step:

* Model 0: [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) with TF-IDF encoder (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional neural network
* Model 6: TensorFlow Hub pretrained feature extractor
* Model 7: Same as model 6 with 10% of training data

Standard steps in the modelling with tensorflow:
* Create a model
* Compile the model
* Fit the mdel
* Evaluate the model
* Improve through experimentation

### Modeling 0: Getting a baseline (Naive Bayes with TF-IDF)

Creating a baseline model is so important as with all machine learning modelling experiments for bencmarking.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to integers.

**Note:** It's so common practice to use non-DL(deep learning) algorithms as a baseline because of their speed. Later on, we can use DL algorithm and improve upon the non-DL models.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # convert words to numbers using TF-IDF
    ('clf', MultinomialNB()) # model the text
])

# Fit the pipline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline model's accuracy: %{baseline_score*100:.2f}")

Baseline model's accuracy: %79.27


In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
# First 20 predictions
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time such as:

* Accuracy
* Precision
* Recall
* F1-Score

Let's create a fuction to evaluate our model's prediction with above metrics.

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculate the model accuracy, precision, recall, f1-score of a binary classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred)
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    model_results = {
        'accuracy': model_accuracy,
        'precision': model_precision,
        'recall': model_recall,
        'f1': model_f1
    }
    return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Feed Forward Neural Network (Dense model)

In [ ]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to Tensorboard logs
SAVE_DIR = 'model_logs'

In [ ]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1D string
x = text_vektorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [ ]:
# Get the model summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
# Compile the model
model_1.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [ ]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20230820-194603
Epoch 1/5
215/215 [==============================] - 12s 51ms/step - loss: 0.5015 - accuracy: 0.8373 - val_loss: 0.4848 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3304 - accuracy: 0.8924 - val_loss: 0.4521 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2633 - accuracy: 0.9079 - val_loss: 0.4552 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2232 - accuracy: 0.9203 - val_loss: 0.4720 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1936 - accuracy: 0.9342 - val_loss: 0.4888 - val_accuracy: 0.7835


In [ ]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4888 - accuracy: 0.7835


[0.4888142943382263, 0.7834645509719849]

In [ ]:
# Make some predictions and evaluate the model
model_1_pred_probs = model_1.predict(val_sentences, verbose=0)
model_1_pred_probs.shape

(762, 1)

In [ ]:
# Look at the first 10 prediction
model_1_pred_probs[:10]

array([[0.39744207],
       [0.75707537],
       [0.99823856],
       [0.11548404],
       [0.08644277],
       [0.9536653 ],
       [0.9018233 ],
       [0.9962011 ],
       [0.9791575 ],
       [0.28103557]], dtype=float32)

In [ ]:
# Convert model prediction probabilities to labeled format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculate the model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 0.7834645669291339,
 'precision': 0.7875988615317319,
 'recall': 0.7834645669291339,
 'f1': 0.7806382038884261}

In [ ]:
# Remind the baseline results
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Hmmm. It look like our dense model didn't perform as well as we expect.

## Visualizing learned embeddings

In [ ]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vektorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocabulary: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocabulary: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
# Get the weight matrix of embedding layer
# (These are the numberical representations of each token in our training data, which have been learned 5 epochs)
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights)

[[ 0.05334017 -0.13442269  0.14436862 ... -0.09715585 -0.00461644
  -0.01404344]
 [-0.04088932 -0.08356974  0.19279027 ... -0.10066718 -0.04562304
  -0.03027875]
 [ 0.09608827 -0.10568354 -0.00397516 ... -0.08274148 -0.03104976
  -0.0110312 ]
 ...
 [ 0.02530653 -0.02370766 -0.04903965 ... -0.04008653  0.03635004
   0.01889286]
 [-0.00512718 -0.07769205  0.08492303 ...  0.08170798  0.11475199
   0.08057176]
 [ 0.08319083 -0.13506244  0.08801189 ...  0.09580076  0.12878689
   0.1621329 ]]


In [ ]:
# Get the shape of embed weights
embed_weights.shape

(10000, 128)

Now, we have got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, Tensorflow has a handy tool projector: https://projector.tensorflow.org/

And Tensorflow also has an incredible guide on word embeddings themselves.

In [ ]:
# Create embedding files (we got this from Tensorflow's word embeddings documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download files from Colab and upload to Projector
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## Recurrent Neural Networks (RNN's)

RNN's is very useful for sequence data.